In [1]:
import os 
import numpy as np
import pandas as pd 
from datetime import datetime as dt

In [2]:
work_dir = os.getcwd()

In [15]:
test_data = pd.read_csv(os.path.join(work_dir, "data", "output_monthly.csv"))
newDate1 = []
for i in range(len(test_data)):
    newDate1.append(dt.strptime(test_data['Date'][i], "%Y/%m/%d").strftime('%Y/%m/%d'))
test_data["Date"] = newDate1
test_data.head(5)

,Date,FPA,FPXI,IQDE,IQDY,CHIQ,GEM,ADRA,PAF,IEMG,...,MOTI,REMX,VEU,VXUS,AXJL,EMCG,XSOE,DNL,IHDG,DBAP
0,2016/02/01,-0.004065,-0.016344,-0.005467,-0.016940,0.006116,-0.007599,-0.036860,-0.075067,-0.001163,...,-0.013239,0.107356,-0.021908,-0.020621,0.007550,0.004204,0.000000,-0.011648,-0.031325,-0.039225
1,2016/03/01,0.072087,0.084702,0.069227,0.100611,0.062687,0.082933,0.074967,0.009138,0.086590,...,0.062715,0.066208,0.054802,0.053323,0.074796,0.104287,0.085929,0.067153,-0.003218,0.000517
2,2016/04/01,-0.006795,0.013285,0.024041,0.039447,0.020657,0.003231,0.009688,0.134409,0.008554,...,0.036194,0.078076,0.028978,0.031992,0.002956,0.004722,0.003395,0.017115,0.003653,0.062500
3,2016/05/02,-0.037163,-0.013481,-0.022355,-0.016216,-0.036900,-0.029424,0.007104,0.010975,-0.031942,...,-0.036107,-0.010036,-0.014237,-0.016332,-0.023800,-0.028721,-0.041083,-0.022242,0.025641,0.022146
4,2016/06/01,0.032858,-0.037606,0.011693,-0.010040,-0.000974,0.045209,-0.012879,-0.049951,0.043363,...,-0.039088,-0.002132,-0.006479,-0.009248,0.046972,0.053763,-0.008048,0.017751,-0.016843,-0.026882


In [33]:
USG1M = pd.read_csv(os.path.join(work_dir, "Risk_free_rate", "DGS1MO.csv"))

newDate = []
for i in range(len(USG1M)):
    date_str =  str(USG1M['Time'][i])[:4] + '-' + str(USG1M['Time'][i])[4:6] + '-' + str(USG1M['Time'][i])[6:]
    newDate.append(dt.strptime(date_str, '%Y-%m-%d').strftime('%Y/%m/%d'))
USG1M["Date"] = newDate

weekly_return = []
for i in range(len(USG1M)):
    try:
         weekly_return.append(float(USG1M["DGS1MO"][i])/100/12)
    except:
        weekly_return.append(0)
USG1M["DGS1MO"] = weekly_return
USG1M.tail()

,Time,DGS1MO,Date
1300,20190503,0.002017,2019/05/03
1301,20190506,0.002025,2019/05/06
1302,20190507,0.002033,2019/05/07
1303,20190508,0.002017,2019/05/08
1304,20190509,0.002025,2019/05/09


In [34]:
cum_ret = []
last_week_pos = 0
cur_week_pos = 0

for i in range(len(USG1M)):
    
    if(USG1M["Date"][i] in newDate1):
        if(last_week_pos == 0): #base case
            last_week_pos = i
            cur_week_pos = i
            cum_ret.append(np.nan)
        else:
            last_week_pos = cur_week_pos
            cur_week_pos = i
            print(np.nanmean((USG1M['DGS1MO'][last_week_pos:cur_week_pos])))
            cum_ret.append(np.nanmean((USG1M['DGS1MO'][last_week_pos:cur_week_pos])))
    else:
        cum_ret.append(np.nan)

USG1M['Rf'] = cum_ret 
USG1M.head()
new_USG1M = USG1M.loc[:, ["Date","Rf"]]

0.00020317460317460315
0.00020181159420289856
0.00015634920634920635
0.00017992424242424244
0.000184469696969697
0.0002091269841269841
0.00021739130434782612
0.00015227272727272727
0.00019365079365079368
0.00022727272727272727
0.00032210144927536234
0.0003936507936507936
0.00038166666666666666
0.0005521739130434784
0.0005904166666666667
0.0005804347826086957
0.0007011363636363637
0.0007714285714285713
0.0008163043478260868
0.0007888888888888888
0.0007920454545454546
0.0008670454545454547
0.000912121212121212
0.001034090909090909
0.0010929166666666667
0.0013018939393939395
0.001383730158730159
0.0013605072463768117
0.0015043650793650795
0.001506060606060606
0.00155
0.0016973684210526317
0.0017326086956521742
0.0016973484848484854
0.0017026515151515149
0.001909469696969697
0.0019199999999999998


In [36]:
Results = test_data.merge(new_USG1M, on = "Date")
Results.head(5)

,Date,FPA,FPXI,IQDE,IQDY,CHIQ,GEM,ADRA,PAF,IEMG,...,REMX,VEU,VXUS,AXJL,EMCG,XSOE,DNL,IHDG,DBAP,Rf
0,2016/02/01,-0.004065,-0.016344,-0.005467,-0.016940,0.006116,-0.007599,-0.036860,-0.075067,-0.001163,...,0.107356,-0.021908,-0.020621,0.007550,0.004204,0.000000,-0.011648,-0.031325,-0.039225,NaN
1,2016/03/01,0.072087,0.084702,0.069227,0.100611,0.062687,0.082933,0.074967,0.009138,0.086590,...,0.066208,0.054802,0.053323,0.074796,0.104287,0.085929,0.067153,-0.003218,0.000517,0.000203
2,2016/04/01,-0.006795,0.013285,0.024041,0.039447,0.020657,0.003231,0.009688,0.134409,0.008554,...,0.078076,0.028978,0.031992,0.002956,0.004722,0.003395,0.017115,0.003653,0.062500,0.000202
3,2016/05/02,-0.037163,-0.013481,-0.022355,-0.016216,-0.036900,-0.029424,0.007104,0.010975,-0.031942,...,-0.010036,-0.014237,-0.016332,-0.023800,-0.028721,-0.041083,-0.022242,0.025641,0.022146,0.000156
4,2016/06/01,0.032858,-0.037606,0.011693,-0.010040,-0.000974,0.045209,-0.012879,-0.049951,0.043363,...,-0.002132,-0.006479,-0.009248,0.046972,0.053763,-0.008048,0.017751,-0.016843,-0.026882,0.000180


In [19]:
Results.to_csv(os.path.join(work_dir, "data", "output_monthly_with_rf.csv"), index = False)